<a href="https://colab.research.google.com/github/coder-Yingchao/librealsense/blob/master/cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup rendering dependencies for Google Colaboratory.

In [ ]:
!apt-get install -y xvfb ffmpeg > /dev/null 2>&1
!pip install pyvirtualdisplay pygame moviepy > /dev/null 2>&1

Install d3rlpy!

In [ ]:
!pip install d3rlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.4 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827626 sha256=3d0ef9bdb5555330e5cf5a93c5f1680afdf9131d8278098f3ac5e68db6c22dfb
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not cur

Setup cartpole dataset.

In [ ]:
from google.colab import files
uploaded = files.upload()  # 会在左下角弹窗选择文件上传

Saving assembly_offline_env.zip to assembly_offline_env.zip


In [ ]:
!unzip assembly_offline_env.zip

Archive:  assembly_offline_env.zip
   creating: game/
  inflating: game/__init__.py        
   creating: game/__pycache__/
  inflating: game/__pycache__/computer_assembly_game_v1.cpython-311.pyc  
  inflating: game/__pycache__/__init__.cpython-38.pyc  
  inflating: game/__pycache__/__init__.cpython-39.pyc  
  inflating: game/__pycache__/computer_assembly_game.cpython-38.pyc  
  inflating: game/__pycache__/computer_assembly_game_v1.cpython-38.pyc  
  inflating: game/__pycache__/computer_assembly_game_v1.cpython-39.pyc  
  inflating: game/__pycache__/computer_assembly_game_v2.cpython-311.pyc  
  inflating: game/__pycache__/computer_assembly_game_v2.cpython-38.pyc  
  inflating: game/__pycache__/computer_assembly_game_v2.cpython-39.pyc  
  inflating: game/__pycache__/computer_assembly_game_v4.cpython-311.pyc  
  inflating: game/__pycache__/computer_assembly_game_v3_1.cpython-311.pyc  
  inflating: game/__pycache__/__init__.cpython-311.pyc  
  inflating: game/__pycache__/computer_assembly_

In [ ]:
import sys
sys.path.append("/content/gym_env")
import computer_assembly_env_v13  # Ensure this matches the path to your Gym environment
import gymnasium as gym
env = gym.make('ComputerAssemblyEnv-v13')
observation_shape = env.observation_space.shape or env.observation_space.n,
action_shape = env.action_space.n



/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:485: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(


(array([1, 1, 1, 1, 1, 1, 1, 1], dtype=int32), {})

In [ ]:
import h5py
import numpy as np

# 假设文件名叫 "my_data.hdf5"
with h5py.File("/content/data_10_epo.hdf5", "r") as f:
    observations = f["obs"][:]  # shape: (T, obs_dim)
    actions = f["act"][:]            # shape: (T, act_dim) 或 (T,) (离散动作)
    rewards = f["rew"][:]            # shape: (T,)
    terminals = f["terminated"][:]        # shape: (T,)

KeyError: "Unable to synchronously open object (object 'obs' doesn't exist)"

In [ ]:
import d3rlpy

# get CartPole dataset
dataset, env = d3rlpy.datasets.get_cartpole()

Setup data-driven deep reinforcement learning algorithm.

In [ ]:
# setup CQL algorithm
cql = d3rlpy.algos.DiscreteCQLConfig().create(device='cuda:0')

# start training
cql.fit(
    dataset,
    n_steps=10000,
    n_steps_per_epoch=1000,
    evaluators={
        'environment': d3rlpy.metrics.EnvironmentEvaluator(env), # evaluate with CartPole-v1 environment
    },
)

Record video!

In [ ]:
import gym
from gym.wrappers import RecordVideo

# start virtual display
d3rlpy.notebook_utils.start_virtual_display()

# wrap RecordVideo wrapper
env = RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"), './video')

# evaluate
d3rlpy.metrics.evaluate_qlearning_with_environment(cql, env)

Let's see how it works!

In [ ]:
d3rlpy.notebook_utils.render_video("video/rl-video-episode-0.mp4")